## This Notebook creates network patch files to similate traffic routing through Terminal D garage, 
## given Terminal D curbside closure
We will simulate curbside drop off through the garage by significantly reducing the speed at the curb,
and allowing drop/pick up on any lane. This notebook creates patch files that reduces the speed on terminal D curbside,
to simulate maneovering through the garage.

In [1]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element
from lxml import etree
import pandas as pd

In [4]:
# First Read Network file with xml
net_file = 'DFW.net.xml'
path_to_net_file = '../Example_Files/SUMO_Input_Files/' + net_file

In [5]:
net_tree = etree.parse(path_to_net_file)  # parse the network xml file

In [9]:
# Read edges that are not internal
edges = net_tree.xpath('.//edge[not(@function="internal")]')

#### Create Data Frame for all Road Ids

In [19]:
df = pd.DataFrame(columns = ['Link_Id', 'speed'])

In [20]:
i = 0
for e in edges:
    num_lanes = len(e.xpath('.//speed'))
    df.loc[i] = [e.attrib['id'], int(num_lanes)]
    i = i+1

In [21]:
df.head()

,Link_Id,speed
0,-10027460#0,0
1,-10027460#1,0
2,-10027476,0
3,-10031443#0,0
4,-10031443#1,0


### Create patch file for terminal D

In [14]:
# The function is created to be general enough for any terminal. The speed new speed is given in mph,
# and will need to be converted in m/s for SUMO
# def modify_edge_speed(new_speed = 5, terminal = 'D'):
    new_speed = 5
    terminal = 'D'
    m_s_speed = new_speed * 0.44704
    
    # Access all links that have terminal name in their id
    link_ids = pd.Series(df.Link_Id.values.flatten())
    term_links = link_ids[link_ids.str.startswith(terminal)]
    print(term_links)

2671    D_arrive_1
2672    D_arrive_2
2673    D_depart_1
2674    D_depart_2
2675     D_service
dtype: object


In [16]:
    # Initialize xml element
    xedges = Element('edges')
    for l in term_links:
        x = df.loc[df['Link_Id'] == l]
        speed = x['speed'].values[0] # gets the number of lanes
        print("Edge ", x, " old speed: ", speed)
#         xedge = Element ('edge')
#         xedges.append(xedge)
#         xedge.set('id', l)
#         xedge.set('speed', m_s_speed)

KeyError: 'speed'